## 上一个代码的输出

In [1]:
sqls = [{'question_type': 'disease_acompany',
  'sql': ["MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where m.name = '耳鸣' return m.name, r.name, n.name",
   "MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where m.name = '百日咳' return m.name, r.name, n.name",
   "MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where n.name = '耳鸣' return m.name, r.name, n.name",
   "MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where n.name = '百日咳' return m.name, r.name, n.name"]},
 {'question_type': 'disease_do_food',
  'sql': ["MATCH (m:Disease)-[r:do_eat]->(n:Food) where m.name = '耳鸣' return m.name, r.name, n.name",
   "MATCH (m:Disease)-[r:do_eat]->(n:Food) where m.name = '百日咳' return m.name, r.name, n.name",
   "MATCH (m:Disease)-[r:recommand_eat]->(n:Food) where m.name = '耳鸣' return m.name, r.name, n.name",
   "MATCH (m:Disease)-[r:recommand_eat]->(n:Food) where m.name = '百日咳' return m.name, r.name, n.name"]}]

In [2]:
from py2neo import Graph

## 连接图数据库

In [3]:
g = Graph(auth=("neo4j", "neo4jneo4j"))

In [4]:
num_limit = 20

In [5]:
final_answers = []

In [6]:
sqls

[{'question_type': 'disease_acompany',
  'sql': ["MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where m.name = '耳鸣' return m.name, r.name, n.name",
   "MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where m.name = '百日咳' return m.name, r.name, n.name",
   "MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where n.name = '耳鸣' return m.name, r.name, n.name",
   "MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where n.name = '百日咳' return m.name, r.name, n.name"]},
 {'question_type': 'disease_do_food',
  'sql': ["MATCH (m:Disease)-[r:do_eat]->(n:Food) where m.name = '耳鸣' return m.name, r.name, n.name",
   "MATCH (m:Disease)-[r:do_eat]->(n:Food) where m.name = '百日咳' return m.name, r.name, n.name",
   "MATCH (m:Disease)-[r:recommand_eat]->(n:Food) where m.name = '耳鸣' return m.name, r.name, n.name",
   "MATCH (m:Disease)-[r:recommand_eat]->(n:Food) where m.name = '百日咳' return m.name, r.name, n.name"]}]

In [7]:
'''根据对应的qustion_type，调用相应的回复模板'''
def answer_prettify(question_type, answers):
    final_answer = []
    if not answers:
        return ''
    if question_type == 'disease_symptom':
        desc = [i['n.name'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}的症状包括：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'symptom_disease':
        desc = [i['m.name'] for i in answers]
        subject = answers[0]['n.name']
        final_answer = '症状{0}可能染上的疾病有：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_cause':
        desc = [i['m.cause'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}可能的成因有：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_prevent':
        desc = [i['m.prevent'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}的预防措施包括：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_lasttime':
        desc = [i['m.cure_lasttime'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}治疗可能持续的周期为：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_cureway':
        desc = [';'.join(i['m.cure_way']) for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}可以尝试如下治疗：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_cureprob':
        desc = [i['m.cured_prob'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}治愈的概率为（仅供参考）：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_easyget':
        desc = [i['m.easy_get'] for i in answers]
        subject = answers[0]['m.name']

        final_answer = '{0}的易感人群包括：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_desc':
        desc = [i['m.desc'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0},熟悉一下：{1}'.format(subject,  '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_acompany':
        desc1 = [i['n.name'] for i in answers]
        desc2 = [i['m.name'] for i in answers]
        subject = answers[0]['m.name']
        desc = [i for i in desc1 + desc2 if i != subject]
        final_answer = '{0}的症状包括：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_not_food':
        desc = [i['n.name'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}忌食的食物包括有：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_do_food':
        do_desc = [i['n.name'] for i in answers if i['r.name'] == '宜吃']
        recommand_desc = [i['n.name'] for i in answers if i['r.name'] == '推荐食谱']
        subject = answers[0]['m.name']
        final_answer = '{0}宜食的食物包括有：{1}\n推荐食谱包括有：{2}'.format(subject, ';'.join(list(set(do_desc))[:num_limit]), ';'.join(list(set(recommand_desc))[:num_limit]))

    elif question_type == 'food_not_disease':
        desc = [i['m.name'] for i in answers]
        subject = answers[0]['n.name']
        final_answer = '患有{0}的人最好不要吃{1}'.format('；'.join(list(set(desc))[:num_limit]), subject)

    elif question_type == 'food_do_disease':
        desc = [i['m.name'] for i in answers]
        subject = answers[0]['n.name']
        final_answer = '患有{0}的人建议多试试{1}'.format('；'.join(list(set(desc))[:num_limit]), subject)

    elif question_type == 'disease_drug':
        desc = [i['n.name'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}通常的使用的药品包括：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'drug_disease':
        desc = [i['m.name'] for i in answers]
        subject = answers[0]['n.name']
        final_answer = '{0}主治的疾病有{1},可以试试'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'disease_check':
        desc = [i['n.name'] for i in answers]
        subject = answers[0]['m.name']
        final_answer = '{0}通常可以通过以下方式检查出来：{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    elif question_type == 'check_disease':
        desc = [i['m.name'] for i in answers]
        subject = answers[0]['n.name']
        final_answer = '通常可以通过{0}检查出来的疾病有{1}'.format(subject, '；'.join(list(set(desc))[:num_limit]))

    return final_answer

In [8]:
for sql_ in sqls:
    question_type = sql_['question_type']
    queries = sql_['sql']
    answers = []
    
    
    for query in queries: # 运行每一条cypher查询语句
        print(query)
        ress = g.run(query).data()
        print(ress)
        print("---------")
        answers += ress
    print(answers)
    final_answer = answer_prettify(question_type, answers)
    print("**********")
    print(final_answer)
    if final_answer:
        final_answers.append(final_answer)

MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where m.name = '耳鸣' return m.name, r.name, n.name
[{'m.name': '耳鸣', 'r.name': '并发症', 'n.name': '咽鼓管阻塞'}]
---------
MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where m.name = '百日咳' return m.name, r.name, n.name
[{'m.name': '百日咳', 'r.name': '并发症', 'n.name': '肺不张'}]
---------
MATCH (m:Disease)-[r:acompany_with]->(n:Disease) where n.name = '耳鸣' return m.name, r.name, n.name
[{'m.name': '偏头痛', 'r.name': '并发症', 'n.name': '耳鸣'}, {'m.name': '鼻咽血管纤维瘤', 'r.name': '并发症', 'n.name': '耳鸣'}, {'m.name': '间断脉冲噪音损伤', 'r.name': '并发症', 'n.name': '耳鸣'}, {'m.name': '潜水性内耳损伤', 'r.name': '并发症', 'n.name': '耳鸣'}, {'m.name': '咽肌痉挛', 'r.name': '并发症', 'n.name': '耳鸣'}, {'m.name': '咽部硬结病', 'r.name': '并发症', 'n.name': '耳鸣'}, {'m.name': '噪声性耳聋', 'r.name': '并发症', 'n.name': '耳鸣'}, {'m.name': '中毒性眩晕', 'r.name': '并发症', 'n.name': '耳鸣'}, {'m.name': '颞下颌关节紊乱综合征', 'r.name': '并发症', 'n.name': '耳鸣'}, {'m.name': '乳突炎', 'r.name': '并发症', 'n.name': '耳鸣'}, {'m.name': '空调病', 'r.name

In [9]:
ress

[{'m.name': '百日咳', 'r.name': '推荐食谱', 'n.name': '黄瓜拌皮丝'},
 {'m.name': '百日咳', 'r.name': '推荐食谱', 'n.name': '黄瓜拌兔丝'},
 {'m.name': '百日咳', 'r.name': '推荐食谱', 'n.name': '黄瓜三丝汤'},
 {'m.name': '百日咳', 'r.name': '推荐食谱', 'n.name': '小黄瓜凉拌面'},
 {'m.name': '百日咳', 'r.name': '推荐食谱', 'n.name': '罗汉果雪耳鸡汤'},
 {'m.name': '百日咳', 'r.name': '推荐食谱', 'n.name': '排骨汤'},
 {'m.name': '百日咳', 'r.name': '推荐食谱', 'n.name': '百合双耳鸡蛋羹'},
 {'m.name': '百日咳', 'r.name': '推荐食谱', 'n.name': '清蒸鸡蛋羹'}]

In [10]:
final_answer

'耳鸣宜食的食物包括有：樱桃番茄;鸡翅;芝麻;小白菜;南瓜子仁;圆白菜;腰果\n推荐食谱包括有：冬菇油菜心;小黄瓜凉拌面;黄瓜拌兔丝;紫菜鸡蛋莲草汤;黄瓜拌皮丝;黄瓜三丝汤;百合双耳鸡蛋羹;紫菜芙蓉汤;可乐鸡翅;罗汉果雪耳鸡汤;栗子鸡翅;油豆腐油菜;羊肉汤面;清蒸鸡蛋羹;乌药羊肉汤;排骨汤'

In [11]:
final_answers

['耳鸣的症状包括：上干型胸廓出口综合征；颞下颌关节病；颞下颌关节紊乱综合征；空调病；脑血管痉挛；咽肌痉挛；潜水性内耳损伤；药物性耳聋；中毒性眩晕；脑鸣；肺不张；听力障碍；乳突炎；间断脉冲噪音损伤；四肢不用；头风病；肾虚腰痛；咽鼓管阻塞；咽部硬结病；噪声性耳聋',
 '耳鸣宜食的食物包括有：樱桃番茄;鸡翅;芝麻;小白菜;南瓜子仁;圆白菜;腰果\n推荐食谱包括有：冬菇油菜心;小黄瓜凉拌面;黄瓜拌兔丝;紫菜鸡蛋莲草汤;黄瓜拌皮丝;黄瓜三丝汤;百合双耳鸡蛋羹;紫菜芙蓉汤;可乐鸡翅;罗汉果雪耳鸡汤;栗子鸡翅;油豆腐油菜;羊肉汤面;清蒸鸡蛋羹;乌药羊肉汤;排骨汤']